In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
fileroot='2024_03_08_TRF_Maleen/'
filename='2024_03_08_TRF_Maleen_01'

datapath='/home/maleen/rosbags/Transformers/datasets/training/'

# Load the DataFrame from the pickle file
df1 = pd.read_pickle(datapath + filename + '.pkl')

arrays = [np.array(item) for item in df1['Skeleton_3D']]
timestamps = [np.array(item) for item in df1['Skeleton_Timestamp']]

# Convert datetime to seconds from start
timestamps = (timestamps - timestamps[0])

# Stack these arrays along a new axis to create a 3D NumPy array
# Each "slice" of this 3D array represents one frame of keypoints
skeleton_3d_frames = np.stack(arrays, axis=0)

In [ ]:
#Create masks for the data (1 = data present, 0 = data missing)
# Correcting the mask values
masks = np.where(np.isnan(skeleton_3d_frames).any(axis=2), 0, 1)  # 0 for missing, 1 for present

In [ ]:
# Calculate the differences in position and time
position_diff = np.diff(skeleton_3d_frames, axis=0)
time_diff = np.diff(timestamps)

# Ensure that time_diff is of shape (n,1,1) so that it broadcasts correctly when dividing
time_diff = time_diff[:, np.newaxis, np.newaxis]

# Update masks to match the dimensions of position_diff and time_diff
# We use the bitwise AND operator to ensure that both the current and previous frames are valid
masks_pos = masks[:-1, :] & masks[1:, :]

# Add an additional dimension to masks
masks_pos = masks_pos[:,:,np.newaxis]

# Now we calculate velocity, handling missing data according to the mask
# Where the mask is False, we will get np.nan
skel_vel = np.where(masks_pos, position_diff / time_diff, np.nan)

masks_velocity = masks_pos[:-1, :] & masks_pos[1:, :]

# Calculate the differences in velocity
velocity_diff = np.diff(skel_vel, axis=0)

# Now we calculate acceleration, handling missing data according to the mask
# Where the mask is False, we will get np.nan
skel_acc = np.where(masks_velocity, velocity_diff / time_diff[:-1, :, :], np.nan)  # Use time_diff with one less time dimension

In [ ]:
# Now, slice skeleton_3d_frames and skel_vel to match the dimensions of skell_acc
skel_pos= skeleton_3d_frames[2:]
skel_vel = skel_vel[1:]
masks = masks[2:]

In [ ]:
# Initialize arrays to hold the normalized data, medians, and IQRs
norm_pos = np.empty_like(skel_pos)
medians_per_joint_axis_skel_pos = np.empty((skel_pos.shape[1], skel_pos.shape[2]))
iqrs_per_joint_axis_skel_pos = np.empty((skel_pos.shape[1], skel_pos.shape[2]))

norm_vel= np.empty_like(skel_vel)
medians_per_joint_axis_vel = np.empty((skel_vel.shape[1], skel_vel.shape[2]))
iqrs_per_joint_axis_vel = np.empty((skel_vel.shape[1], skel_vel.shape[2]))

norm_acc = np.empty_like(skel_acc)
medians_per_joint_axis_acc = np.empty((skel_acc.shape[1], skel_acc.shape[2]))
iqrs_per_joint_axis_acc = np.empty((skel_acc.shape[1], skel_acc.shape[2]))

def robust_normalize_data_with_clipping(data, masks, medians_per_joint_axis, iqrs_per_joint_axis, normalized_data, clipping_percentiles=(1, 99)):
    for joint in range(data.shape[1]):  # For each joint
        for axis in range(data.shape[2]):  # For each axis (x, y, z)
            joint_axis_data = data[:, joint, axis]
            mask_for_joint = masks[:, joint]

            # Select valid data based on the mask
            valid_data = joint_axis_data[mask_for_joint == 1]

            # Determine clipping thresholds based on percentiles
            lower_threshold, upper_threshold = np.percentile(valid_data, clipping_percentiles) if valid_data.size > 0 else (np.nan, np.nan)

            # Clip the data based on valid mask and thresholds
            clipped_values = np.clip(joint_axis_data, lower_threshold, upper_threshold)

            # Calculate median and IQR for clipped data
            median = np.median(clipped_values[mask_for_joint == 1]) if np.any(mask_for_joint == 1) else np.nan
            q75, q25 = np.percentile(clipped_values[mask_for_joint == 1], [75 ,25]) if np.any(mask_for_joint == 1) else (np.nan, np.nan)
            iqr = q75 - q25

            # Store the calculated medians and IQRs
            medians_per_joint_axis[joint, axis] = median
            iqrs_per_joint_axis[joint, axis] = iqr

            # Normalize the clipped data, avoiding division by zero
            if iqr > 0:
                normalized_values = (clipped_values - median) / iqr
            else:
                normalized_values = clipped_values  # Keep original values if IQR is 0 or nan

            # Apply normalization only where data is present
            normalized_data[:, joint, axis] = np.where(mask_for_joint == 1, normalized_values, np.nan)
            
    return normalized_data, medians_per_joint_axis, iqrs_per_joint_axis

# Example usage with your data
# Note: masks, skel_pos, skel_vel, skel_acc should be defined in your context

norm_pos, medians_pos, iqrs_pos = robust_normalize_data_with_clipping(skel_pos, masks, medians_per_joint_axis_skel_pos, iqrs_per_joint_axis_skel_pos, norm_pos)
norm_vel, medians_vel, iqrs_vel = robust_normalize_data_with_clipping(skel_vel, masks, medians_per_joint_axis_vel, iqrs_per_joint_axis_vel, norm_vel)
norm_acc, medians_acc, iqrs_acc = robust_normalize_data_with_clipping(skel_acc, masks, medians_per_joint_axis_acc, iqrs_per_joint_axis_acc, norm_acc)


In [ ]:
data=skel_acc
norm_data=norm_acc
# Plot the original and normalized data for a specific joint and axis
joint, axis = 0, 0  # Change as needed
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data[:, joint, axis], bins=20, alpha=0.7, label='Original')
plt.title("Original Data Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(norm_data[:, joint, axis], bins=20, alpha=0.7, label='Normalized')
plt.title("Normalized Data Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

# Check the median and range of the normalized data
normalized_median = np.nanmedian(norm_data[:, joint, axis])
print("Median of normalized data:", normalized_median)

within_iqr = ((norm_data[:, joint, axis] > -2) & (norm_data[:, joint, axis] < 2)).sum()
print(f"Data points within [-1, 1] (IQR): {within_iqr} out of {norm_data.shape[0]}")

within_iqr2 = ((data[:, joint, axis] > -1) & (data[:, joint, axis] < 1)).sum()
print(f"Data points within [-1, 1] (IQR): {within_iqr2} out of {norm_data.shape[0]}")

In [ ]:
class SkeletalInputEmbedding(nn.Module):
    def __init__(self, num_joints=18, dof=3, embed_dim=128, seq_len=60):
        super().__init__()
        self.num_joints = num_joints
        self.dof = dof
        self.seq_len = seq_len
        self.embed_dim = embed_dim

        self.joint_embed = nn.Linear(dof, embed_dim)
        self.vel_embed = nn.Linear(dof, embed_dim)
        self.acc_embed = nn.Linear(dof, embed_dim)

        self.register_buffer('positional_encoding', self.get_sinusoidal_encoding(seq_len * num_joints, embed_dim))

    def forward(self, joint_positions, velocities, accelerations, mask=None):
        joint_positions = joint_positions.view(-1, self.seq_len * self.num_joints, self.dof)
        velocities = velocities.view(-1, self.seq_len * self.num_joints, self.dof)
        accelerations = accelerations.view(-1, self.seq_len * self.num_joints, self.dof)

        if mask is not None:
            joint_positions = joint_positions * mask.view(-1, self.seq_len * self.num_joints, 1)
            velocities = velocities * mask.view(-1, self.seq_len * self.num_joints, 1)
            accelerations = accelerations * mask.view(-1, self.seq_len * self.num_joints, 1)

        joint_embeddings = self.joint_embed(joint_positions)
        vel_embeddings = self.vel_embed(velocities)
        acc_embeddings = self.acc_embed(accelerations)

        combined_embeddings = joint_embeddings + vel_embeddings + acc_embeddings
        combined_embeddings += self.positional_encoding[:combined_embeddings.size(1), :].unsqueeze(0)
        combined_embeddings = combined_embeddings.view(-1, self.seq_len, self.num_joints, self.embed_dim)

        return combined_embeddings

    def get_sinusoidal_encoding(self, seq_len, embed_dim):
        pe = torch.zeros(seq_len, embed_dim)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(np.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe


In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, num_layers, dim_feedforward, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x, mask=None):

        # Reshape the input tensor to shape (seq_len, batch_size * num_joints, embed_dim)
        batch_size, seq_len, num_joints, _ = x.shape
        x = x.reshape(seq_len, batch_size * num_joints, self.embed_dim)

        if mask is not None:
            # Permute the mask tensor to shape (batch_size, seq_len, num_joints)
            mask = mask.permute(0, 1, 2)

            # Reshape the mask tensor to shape (batch_size * num_joints, seq_len)
            mask = mask.reshape(batch_size * num_joints, seq_len)

            # Create the padding mask
            padding_mask = mask.eq(0)

            # Pass the input and padding mask to the transformer encoder
            x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
        else:
            x = self.transformer_encoder(x)

        # Reshape the output tensor back to shape (batch_size, seq_len, num_joints, embed_dim)
        x = x.reshape(batch_size, seq_len, num_joints, self.embed_dim)

        return x

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, embed_dim, num_heads, num_layers, dim_feedforward, num_joints, dof, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.num_joints = num_joints
        self.dof = dof

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, dof)

    def forward(self, tgt, memory, num_frames_to_predict, tgt_mask=None, memory_mask=None):
        batch_size, seq_len, num_joints, _ = tgt.shape

        # Reshape the target tensor to shape (seq_len, batch_size * num_joints, embed_dim)
        tgt = tgt.reshape(seq_len, batch_size * num_joints, self.embed_dim)

        # Reshape the memory tensor to shape (seq_len, batch_size * num_joints, embed_dim)
        memory = memory.reshape(seq_len, batch_size * num_joints, self.embed_dim)

        if tgt_mask is not None:
            # Reshape the target mask tensor to shape (batch_size * num_joints, seq_len)
            tgt_mask = tgt_mask.reshape(batch_size * num_joints, seq_len)
            # Create the target padding mask
            tgt_padding_mask = tgt_mask.eq(0)
        else:
            tgt_padding_mask = None

        if memory_mask is not None:
            # Reshape the memory mask tensor to shape (batch_size * num_joints, seq_len)
            memory_mask = memory_mask.reshape(batch_size * num_joints, seq_len)
            # Create the memory padding mask
            memory_padding_mask = memory_mask.eq(0)
        else:
            memory_padding_mask = None

        # Pass the target, memory, and masks to the transformer decoder
        output = self.transformer_decoder(tgt, memory, tgt_key_padding_mask=tgt_padding_mask,
                                          memory_key_padding_mask=memory_padding_mask)

        # Apply a linear transformation to the output
        output = self.fc(output)

        # Reshape the output tensor to shape (batch_size, seq_len, num_joints, dof)
        output = output.reshape(batch_size, seq_len, num_joints, self.dof)

        return output

In [ ]:

def generate_sequences(norm_pos, norm_vel, norm_acc, mask, input_length=60, predict_length=60):
    num_frames = norm_pos.shape[0]
    num_joints = norm_pos.shape[1]
    
    # Calculate the total number of sequences we can create
    num_sequences = num_frames - input_length - predict_length + 1

    # Initialize arrays to store the input and target sequences
    X_pos = np.zeros((num_sequences, input_length, num_joints, 3))
    X_vel = np.zeros((num_sequences, input_length, num_joints, 3))
    X_acc = np.zeros((num_sequences, input_length, num_joints, 3))
    Y_pos = np.zeros((num_sequences, predict_length, num_joints, 3))
    Y_vel = np.zeros((num_sequences, predict_length, num_joints, 3))
    Y_acc = np.zeros((num_sequences, predict_length, num_joints, 3))
    X_mask = np.zeros((num_sequences, input_length, num_joints))
    Y_mask = np.zeros((num_sequences, predict_length, num_joints))

    # Create sequences
    for i in range(num_sequences):
        X_pos[i] = norm_pos[i:i+input_length]
        X_vel[i] = norm_vel[i:i+input_length]
        X_acc[i] = norm_acc[i:i+input_length]
        Y_pos[i] = norm_pos[i+1:i+1+predict_length]
        Y_vel[i] = norm_vel[i+1:i+1+predict_length]
        Y_acc[i] = norm_acc[i+1:i+1+predict_length]
        X_mask[i] = mask[i:i+input_length]
        Y_mask[i] = mask[i+1:i+1+predict_length]

    return X_pos, X_vel, X_acc, X_mask, Y_pos, Y_vel, Y_acc, Y_mask

# Use the function
X_pos, X_vel, X_acc, X_mask, Y_pos, Y_vel, Y_acc, Y_mask = generate_sequences(norm_pos, norm_vel, norm_acc, masks)


In [ ]:
def forward_hook(module, input, output):
    if torch.isnan(output).any():
        print(f"NaN detected in output of {module.__class__.__name__}")

def backward_hook(module, grad_input, grad_output):
    if any(torch.isnan(g).any() for g in grad_input if g is not None):
        print(f"NaN detected in grad input of {module.__class__.__name__}")
    if any(torch.isnan(g).any() for g in grad_output if g is not None):
        print(f"NaN detected in grad output of {module.__class__.__name__}")

def register_hooks(model):
    for name, module in model.named_modules():
        module.register_forward_hook(forward_hook)
        module.register_backward_hook(backward_hook)


class TransformerModel(nn.Module):
    def __init__(self, num_joints=18, dof=3, embed_dim=128, num_heads=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=512, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_embedding = SkeletalInputEmbedding(num_joints=num_joints, dof=dof, embed_dim=embed_dim)
        self.encoder = TransformerEncoder(embed_dim=embed_dim, num_heads=num_heads, num_layers=num_encoder_layers, dim_feedforward=dim_feedforward, dropout=dropout)
        self.decoder = TransformerDecoder(embed_dim=embed_dim, num_heads=num_heads, num_layers=num_decoder_layers, dim_feedforward=dim_feedforward, num_joints=num_joints, dof=dof, dropout=dropout)
        self._initialize_weights()

    def _initialize_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
                if module.bias is not None:
                    torch.nn.init.constant_(module.bias, 0)

    def forward(self, pos, vel, acc, mask):
        embeddings = self.input_embedding(pos, vel, acc, mask)
        # if torch.isnan(embeddings).any():
        #     print("NaN detected in embeddings")

        encoder_output = self.encoder(embeddings)
        # if torch.isnan(encoder_output).any():
        #     print("NaN detected in encoder output")

        predicted_output = self.decoder(encoder_output, encoder_output, num_frames_to_predict=60)
        # if torch.isnan(predicted_output).any():
        #     print(predicted_output)
        
        return predicted_output

# Convert to PyTorch tensors
X_pos_tensor = torch.tensor(X_pos, dtype=torch.float32)
X_vel_tensor = torch.tensor(X_vel, dtype=torch.float32)
X_acc_tensor = torch.tensor(X_acc, dtype=torch.float32)
X_mask_tensor = torch.tensor(X_mask, dtype=torch.bool)

Y_pos_tensor = torch.tensor(Y_pos, dtype=torch.float32)

dataset = TensorDataset(X_pos_tensor, X_vel_tensor, X_acc_tensor, X_mask_tensor, Y_pos_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Initialize the model
model = TransformerModel()
model.cuda()  # Assuming you're using GPU
register_hooks(model)

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop

best_loss = float('inf')
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for pos, vel, acc, mask, y_pos in dataloader:
        pos, vel, acc, mask = pos.cuda(), vel.cuda(), acc.cuda(), mask.cuda()
        y_pos = y_pos.cuda()
    
        optimizer.zero_grad()
        output = model(pos, vel, acc, mask)
        #print(output)
    
        mask = mask.unsqueeze(-1).expand_as(output)
        output = output.where(~torch.isnan(output), torch.zeros_like(output))
        y_pos = y_pos.where(~torch.isnan(y_pos), torch.zeros_like(y_pos))
        
        masked_output = output * mask
        masked_y_pos = y_pos * mask
        
        loss = criterion(masked_output, masked_y_pos)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()

    for name, param in model.named_parameters():
        if torch.isnan(param).any():
            print(f"Pre-save NaN found in {name}")
        else:
            print(f"{name} - max: {param.max()}, min: {param.min()}, mean: {param.mean()}")

    epoch_loss = running_loss / len(dataloader)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model_weights.pth')  # Save the best model weights
        print(f'Epoch {epoch+1}, Loss: {epoch_loss} (new best, model saved)')
    else:
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

print("Training complete")

In [ ]:
# Example of loading weights
try:
    model.load_state_dict(torch.load('best_model_weights.pth'))
    print("Weights loaded successfully.")
except Exception as e:
    print("Failed to load weights:", e)


# Check for NaNs or extreme values in weights
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print(f"NaN found in {name}")
    if param.abs().max() > 1e6:  # Example threshold for "extreme" values
        print(f"Extreme values found in {name}")





In [ ]:
def test_transformer_encoder():
    model = TransformerEncoder(embed_dim=128, num_heads=8, num_layers=3, dim_feedforward=512)
    batch_size, seq_len, num_joints = 2, 10, 5
    x = torch.randn(batch_size, seq_len, num_joints, 128)  # Simulate input
    mask = torch.randint(0, 2, (batch_size, seq_len, num_joints), dtype=torch.bool)  # Simulate binary mask

    output = model(x, mask)
    assert output.shape == (batch_size, seq_len, num_joints, 128), "Output shape mismatch"

test_transformer_encoder()

In [ ]:
skeleton_3d_frames[0]